Packages

In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBRegressor
import joblib
import matplotlib.pylab as pl
import seaborn as sns
import shap
from scipy.stats import pearsonr

Reading the data

In [2]:
train_data = pd.read_csv("./Data/cleaned_train.csv")
test_data = pd.read_csv("./Data/cleaned_test.csv")
target = pd.read_csv("./Data/cleaned_target.csv")

target.set_index("Id", inplace = True)

Label encoding the features

In [3]:
enc = LabelEncoder()
train_result = {}
test_result = {}
for col in train_data.columns:
    if (train_data[col].dtype == "int" or train_data[col].dtype == "float"):
        train_result[col] = train_data[col]
        test_result[col] = test_data[col]
    else:
        train_result[col] = pd.Series(enc.fit_transform(train_data[col]))
        test_result[col] = pd.Series(enc.transform(test_data[col]))
train_features = pd.DataFrame(train_result)
test_features = pd.DataFrame(test_result)

train_features.set_index("Id", inplace = True)
test_features.set_index("Id", inplace = True)

In [4]:
train_data.columns

Index(['Id', 'LotArea', 'OverallQual', 'GarageCars', 'TotRmsAbvGrd',
       'OverallCond', 'Fireplaces', 'CentralAir', 'Street', 'Alley',
       'YearBuilt', 'YearRemodAdd', 'YrSold', 'KitchenQual', 'ExterQual',
       'ExterCond', 'HeatingQC', 'GarageQual', 'PoolQC', 'BsmtCond',
       'Utilities', 'Fence', 'LandSlope', 'LotShape', 'BsmtExposure',
       'BldgType', 'Functional', 'ProxPos', 'ProxRoad', 'ProxRail', 'Heating',
       'BasementQualFactor', 'TotLivArea', 'FullBath', 'HalfBath', 'PorchSF',
       'Neighborhood', 'MSSubClass', 'SaleCondition'],
      dtype='object')

In [ ]:
train_features.columns

Random Forest

In [ ]:
rf = RandomForestRegressor(n_jobs = -1, n_estimators = 140)
rf_params = {'max_features': range(3,10), 'max_depth': range(1, 13), 'min_samples_leaf': [4, 7, 14, 70, 140]}
gs_rf = GridSearchCV(rf, rf_params, cv = 10, return_train_score = True)

In [ ]:
np.random.seed(0)
%time gs_rf.fit(train_features, target['SalePrice'])

In [ ]:
gs_rf.best_score_

In [ ]:
gs_rf.best_params_

In [ ]:
importances = pd.Series(gs_rf.best_estimator_.feature_importances_, index=train_features.columns).sort_values(ascending=False)
importances[:20]

In [ ]:
model_rf = gs_rf.best_estimator_

XGBoost without early stopping

In [ ]:
xgb = XGBRegressor(n_jobs = -1, booster = "gbtree")
eta_range = [0.01 * i for i in range(1, 21)]
max_depth_range = range(2, 6)
n_estimators = range(100, 301,550)
xgb_params = {'max_depth': max_depth_range, 'eta': eta_range, 'n_estimators' : n_estimators}
gs_xgb = GridSearchCV(xgb, xgb_params, cv = 10, return_train_score = True)

In [ ]:
np.random.seed(0)
%time gs_xgb.fit(train_features, target, verbose = 0)

XGBoost with early stopping

In [5]:
train_features_grid, train_features_validation, target_grid, target_validation = train_test_split(train_features, target, test_size = 0.1)

In [6]:
xgb = XGBRegressor(n_jobs = -1, booster = "gbtree", n_estimators = 1000, colsample_bytree = 0.5)
eta_range = [0.01 * i for i in range(1, 41)]
max_depth_range = range(1, 7)
xgb_params = {'max_depth': max_depth_range, 'eta': eta_range}
gs_xgb = GridSearchCV(xgb, xgb_params, cv = 10, return_train_score = True)

In [7]:
np.random.seed(0)
fit_params={"early_stopping_rounds":10, 
            "eval_metric" : "mae", 
            "eval_set" : [[train_features_validation, target_validation]]}
%time gs_xgb.fit(train_features_grid, target_grid,verbose = 0, **fit_params)

CPU times: user 26min 35s, sys: 3min 35s, total: 30min 11s
Wall time: 4min 41s


GridSearchCV(cv=10,
             estimator=XGBRegressor(base_score=None, booster='gbtree',
                                    colsample_bylevel=None,
                                    colsample_bynode=None, colsample_bytree=0.5,
                                    gamma=None, gpu_id=None,
                                    importance_type='gain',
                                    interaction_constraints=None,
                                    learning_rate=None, max_delta_step=None,
                                    max_depth=None, min_child_weight=None,
                                    missing=nan, monotone_constraints=None,
                                    n_estimators=1000, n_...
                                    num_parallel_tree=None, random_state=None,
                                    reg_alpha=None, reg_lambda=None,
                                    scale_pos_weight=None, subsample=None,
                                    tree_method=None, validate_parame

In [8]:
gs_xgb.best_score_

0.8869885531672438

Second round, narrowing in on eta

In [9]:
xgb = XGBRegressor(n_jobs = -1, booster = "gbtree", n_estimators = 1000, colsample_bytree = 0.5, max_depth = gs_xgb.best_params_["max_depth"])
eta_base = gs_xgb.best_params_["eta"] - 0.01
eta_range = [eta_base + 0.001 * i for i in range(1,20)]
xgb_params = {'eta': eta_range}
gs_xgb = GridSearchCV(xgb, xgb_params, cv = 10, return_train_score = True)

In [10]:
np.random.seed(0)
fit_params={"early_stopping_rounds":10, 
            "eval_metric" : "mae", 
            "eval_set" : [[train_features_validation, target_validation]]}
%time gs_xgb.fit(train_features_grid, target_grid,verbose = 0, **fit_params)

CPU times: user 9min 6s, sys: 1min 15s, total: 10min 21s
Wall time: 1min 35s


GridSearchCV(cv=10,
             estimator=XGBRegressor(base_score=None, booster='gbtree',
                                    colsample_bylevel=None,
                                    colsample_bynode=None, colsample_bytree=0.5,
                                    gamma=None, gpu_id=None,
                                    importance_type='gain',
                                    interaction_constraints=None,
                                    learning_rate=None, max_delta_step=None,
                                    max_depth=4, min_child_weight=None,
                                    missing=nan, monotone_constraints=None,
                                    n_estimators=1000, n_job...
                                    scale_pos_weight=None, subsample=None,
                                    tree_method=None, validate_parameters=None,
                                    verbosity=None),
             param_grid={'eta': [0.011, 0.012, 0.013000000000000001, 0.014,
        

In [11]:
gs_xgb.best_score_

0.8909653149988472

Fitting the residuals

In [ ]:
model_xgb = gs_xgb.best_estimator_
predvalues_train_xgb = pd.Series(model_xgb.predict(train_features))
prediction_train_xgb = pd.DataFrame({"Id" : train_data["Id"], "SalePrice" : predvalues_train_xgb})
prediction_train_xgb.set_index("Id", inplace = True)
residuals = target - prediction_train_xgb

In [ ]:
lm = Ridge()
lm_params = {"alpha" : np.linspace(0.001, 100, 100)}
gs_lm = GridSearchCV(lm, lm_params, cv = 10, return_train_score = True)

In [ ]:
def resid_correl(col):
    return pearsonr(residuals["SalePrice"], col)[0]

In [ ]:
correlations = [resid_correl(train_features[col]) for col in train_features.columns]
sorted_correlations = pd.Series(correlations, index=train_features.columns).sort_values(ascending=False)
mask = (sorted_correlations > 0.1) | (sorted_correlations < -0.1)
selected_cols = sorted_correlations[mask].index

In [ ]:
np.random.seed(0)
%time gs_lm.fit(train_features[selected_cols], residuals)

In [ ]:
gs_lm.best_score_

In [ ]:
gs_lm.best_estimator_.coef_

XGBoost outcomes

In [ ]:
gs_xgb.best_score_

In [12]:
gs_xgb.best_params_

{'eta': 0.019000000000000003}

In [13]:
importances = pd.Series(gs_xgb.best_estimator_.feature_importances_, index=train_features.columns).sort_values(ascending=False)
importances[:20]

OverallQual           0.231610
ExterQual             0.191148
KitchenQual           0.109700
GarageCars            0.108258
TotLivArea            0.106746
Neighborhood          0.051398
FullBath              0.021095
TotRmsAbvGrd          0.019907
Fireplaces            0.018057
YearRemodAdd          0.012275
CentralAir            0.011574
Alley                 0.011148
BsmtExposure          0.011110
ProxPos               0.008331
LotArea               0.008167
BldgType              0.007794
BasementQualFactor    0.007233
YearBuilt             0.007129
MSSubClass            0.007031
HalfBath              0.005682
dtype: float32

Running the model on the test dataset

In [ ]:
pred_xgb = pd.Series(model_xgb.predict(test_features))
prediction_xgb = pd.DataFrame({"Id" : test_data["Id"], "SalePrice" : pred_xgb})
prediction_xgb.set_index("Id", inplace = True)

In [ ]:
prediction_xgb.to_csv("./Data/prediction_xgb.csv")

In [ ]:
pred_rf = pd.Series(model_rf.predict(test_features))
prediction_rf = pd.DataFrame({"Id" : test_data["Id"], "SalePrice" : pred_rf})
prediction_rf.set_index("Id", inplace = True)

In [ ]:
prediction_rf.to_csv("./Data/prediction_rf.csv")

In [ ]:
pred_av = (pred_xgb + pred_rf) / 2
prediction_av = pd.DataFrame({"Id" : test_data["Id"], "SalePrice" : pred_av})
prediction_av.set_index("Id", inplace = True)

In [ ]:
prediction_av.to_csv("./Data/prediction_av.csv")

Save encoder and model for later

In [ ]:
filename = 'fitted_model.sav'
joblib.dump(model_xgb, filename)
filename = 'fitted_encoder.sav'
joblib.dump(enc, filename)

In [ ]:
train_features.to_csv("./Data/encoded_train_data.csv")

In [ ]:
shap_values = shap.TreeExplainer(model_xgb).shap_values(train_features)

In [ ]:
shap.summary_plot(shap_values, train_features)

In [ ]:
shap.dependence_plot("TotLivArea", shap_values, train_features)

In [ ]:
shap.dependence_plot("OverallQual", shap_values, train_features)

In [ ]:
shap.dependence_plot("LotArea", shap_values, train_features)

In [ ]:
shap.dependence_plot("Neighborhood", shap_values, train_features)

In [ ]:
shap_interaction_values = shap.TreeExplainer(model_xgb).shap_interaction_values(train_features)

In [ ]:
shap.summary_plot(shap_interaction_values, train_features)

In [ ]:
shap.dependence_plot(
    ("TotLivArea", "TotLivArea"),
    shap_interaction_values, train_features,
    display_features = train_features
)

In [ ]:
shap.dependence_plot(
    ("TotLivArea", "OverallQual"),
    shap_interaction_values, train_features,
    display_features = train_features
)

In [ ]:
shap.dependence_plot(
    ("Neighborhood", "OverallQual"),
    shap_interaction_values, train_features,
    display_features = train_features
)

In [ ]:
tmp = np.abs(shap_interaction_values).sum(0)
for i in range(tmp.shape[0]):
    tmp[i,i] = 0
inds = np.argsort(-tmp.sum(0))[:50]
tmp2 = tmp[inds,:][:,inds]
pl.figure(figsize=(12,12))
pl.imshow(tmp2)
pl.yticks(range(tmp2.shape[0]), train_features.columns[inds], rotation=50.4, horizontalalignment="right")
pl.xticks(range(tmp2.shape[0]), train_features.columns[inds], rotation=50.4, horizontalalignment="left")
pl.gca().xaxis.tick_top()
pl.show()

In [ ]:
def heatmap(valuesDF, threshold):
    # Make a mask to only show the lower left part of the table
    mask = np.triu(np.ones_like(valuesDF, dtype = bool))

    cmap = sns.diverging_palette(220, 20, as_cmap = True)

    # Draw the heatmap with the mask and correct aspect ratio
    visual = sns.heatmap(valuesDF, mask = mask, cmap = cmap, center = threshold,
                square = True, linewidths = .5, annot = True);

    return visual